Did someone test Omen of CLarity procrate on classic betas ? Wowheadclassic suggests that it's 2 ppm, but I thought the ppm mechanic was only introduced in TBC.
Technically, 2 ppm is pretty insane for a leveling feral druid. That's 2 free regrowth per minute. 42 seconds of free hot and 2 medium heals.
More realistically, that's still around 1.2 to 1.6 proc per minute considering misses and moving around.


Valkeneth: so if white damage is like 40%, yellow 60%, u get 50% speed on the white so 60 + 60 = 120,   20% increase on your damage while MCP is up, assuming omen of clarity procs per minute.. for every 10% of white damage you have, you will get a 5% increase on the total basically

5:40 PM] FFluidity: Thick hide can still be useful if you are concerned about using enrage and suffering from the armor loss even if you are armor capped without it..

[5:41 PM] Fluidity: PPM is really misunderstood by a lot including me as far as what it actually means in vanilla..  @Scylla did a pretty good job explaining it a few weeks back and provided some additional linkage to understand what it really means

[5:43 PM] Fluidity: You can basically use the PPM to figure out a percentage based on your weapon speed and increase proc rate through haste mechanics and exceed the average of 1 every 30 seconds

[5:44 PM] Fluidity: Also bare in mind at the beginning of tbc ooc had a hidden internal cooldown of 10 seconds I dont know when that was added but it got changed in tbc

6:04 PM] Fluidity: In the next data push, you will find Omen of Clarity has been changed a bit. The 10 second cooldown has been removed, the procs per minute has been raised from 2 to 3.5, and melee abilities no longer trigger it. Spell interaction with Omen of Clarity remains relatively unchanged, with its hidden chance to trigger off spells reduced by half (and rolling the dice about 2x as often).

What this means is that more auto-attacks landing on the target will increase the benefit from Omen of Clarity. Haste, increased hit chance, and expertise will all make Omen of Clarity better.

We are aware of the concerns with reaching the hit rating cap, and will make sure Druids can pick up Rogue items to wear without exceeding that cap. This means Rogues will likely socket for hit, while Druids socket for Strength or Agility.
[6:04 PM] Fluidity: That's the patch before tbc goes live
[6:05 PM] Fluidity: And again if Scylla shows up she can further explain it
[6:07 PM] Fluidity: PPM in itself is a term used to originate describe procs from enchants and has been bastardized by the community

6:10 PM] xkcd44: Scylla06/30/2019
@Fluidity the source is a 2006 wowwiki revision on proc rates and PPM. PPM is essentially what determines the proc rate but there’s no cap on how many times it can proc in a minute. It’s also how it works on all pservers and there’s nothing from the classic beta to suggest it works differently. I can pull up the exact information when I get home
[6:11 PM] xkcd44: ive never seen a server which capped OOC, has it worked diff in beta or something?
[6:11 PM] Fluidity: I was just simply pointing our that ur not hard capped at 2 procs per minute just because it's called 2 PPM
[6:12 PM] Fluidity: That haste and special attacks have the opportunity to proc it and its percent chance of procing is determined by weapon swing speed

[8:25 PM] Scrubnoob: There is most definitely not an internal cooldown on omen of clarity

7:58 PM] Kazparov: What's the Proc rate for Omen of Clarity? I read something like 5-6%

10:13 AM] C'thunprenerf: Faster attacks = more Omen of clarity procs too


2:35 PM] Pokey: oh nah im p sure it does scale based on AS, it's just around 6% in cat form
[2:35 PM] Tarregor: so if its 6% in cat form, then its likely to be 15% in bear form
[2:36 PM] Tarregor: anyway, got a source for that?
[2:36 PM] Tarregor: one that isn't from "vanillawowwiki", which includes TBC+ stuff

2:13 PM] Bearspirit: Do we know the ppm for bear Omen of Clarity? Would it be worth melee weaving for free heals? Say with the regrowth build for example.
[2:14 PM] Untamed: Flat 10%

11:53 PM] Fendral: Both me and Mobia have written monte carlo simulations for feral dps that I think should be more accurate than spreadsheets since we can account for stuff like omen of clarity procs and extra combo points from critting shred, but obviously it takes a lot of simulations to get good accuracy

10:04 AM] Vogz: can Omen of Clarity proc off blocked, missed, dodged, parried attacks?  
[10:18 AM] Azurmen32: @Secondcoming yes  
[10:35 AM] ocd: @Vogz  I tested in the latest version of cMaNGOS as your question got me intruiged to find out. On the emulation software it doesn't seem to procc off of any blocks, misses, dodges or parries. This can be different in Classic ofcourse, but thought I'd share my findings as I couldn't find anything written about it online.  
[10:36 AM] ocd: If anyone knows different from the beta feel free to correct me as I'd like to know as well.

====================================

If you wear a +1% spell hit chance item, the above percentages will increase by 1. +2% gives +2.

Etc...

For binary spells only, there is an additional modifier for the resistance of the victim to your particular spell school: fire, frost, shadow, nature, arcane. That modifier is multiplied by your hit chance to get your actual chance to land. This is done with binary spells only, because they never do partial damage.

Example:

Eyonix the Mage (level 60) fires a frost bolt at Yeti of Doom (level 63). Eyonix is also wearing a total of +6% spell hit gear. Yeti of Doom has frost resistance such that he takes 50% from level 60 frost attacks. So, here's the hit calculation:

0.83 (83% for +3 levels mob) + 0.06 (+6% spell hit) = 0.89

0.89*0.5 (50% damage from frost) = 0.445.

The game will roll a number between 0 and 1, and if it's less than 0.445, the frost bolt will hit for full damage. Otherwise, a resist message will appear.

2nd Example:

After the resist, Eyonix decides to fire a fireball at Yeti of Doom. Eyonix still has +6% spell hit. Fireball is not a binary spell. Here's the calculation:

0.83+0.06= 0.89

The game will roll a number between 0 and 1, and if its less than 0.89, the fireball will hit. Otherwise, a resist message will appear. After the fireball lands, the game will then apply spell resistance to determine a partial resist, if any. Assuming the yeti also has 50% fire resistance, on average, 50% of the damage will be resisted.

I hope that illuminates the impact of +spell hit gear on magical combat.

Since I had to clarify in my post on General, I thought I'd do the same here: Binary spells are any spells that have a damage component and a non-damage effect. (examples: Frost Bolt, Frost Shock). Non-binary spells are spells that do only damage. (examples: Shadow Bolt, Fireball, Wrath).

=====================================================================================================
goede post over spell resist:
https://dwarfpriest.wordpress.com/2008/01/07/spell-hit-spell-penetration-and-resistances/


In [1]:
#Creatures at your level have a 5% chance to Dodge your attacks. 
#Each additional level the target has over the player grants them 0.5% additional chance
#to dodge. (So 6.5% chance to Dodge for creatures 3 levels above the player.)

In [2]:
# Note to self: I haven't implemented the following yet because these kind of auras are out of scope for cat-simulator

# In addition, when fighting +3 level mobs there's a flat modifier placed on your total crit chance gained from auras.
# Auras in this context are talents such as Cruelty or Axe Specialization, gear that directly gives crit % through 
# Equip: effects, buffs that increase you crit chance like Songflower Serenade or Leader of the Pack, and consumables such 
# as Elixir of the Mongoose. It does not include crit gained indirectly through Agility, neither base Agility or 
# Agility from gear.

In [3]:
import pandas as pd
import numpy as np

In [4]:
base_stats_df = pd.read_csv('base_stats.csv', index_col = 0)
base_stats_df['Melee Crit'] = round(base_stats_df['Melee Crit'] / 100,4)
base_stats_df.tail(5)

,Agility,Intellect,Strength,Stamina,Spirit,Attack Power,Melee Crit
Level,,,,,,,
56,62,57,54,56,57,71,0.3854
57,63,58,55,57,58,72,0.3922
58,64,59,56,58,59,73,0.3989
59,64,59,56,58,59,73,0.4057
60,65,100,62,69,110,104,0.0325


In [5]:
#level x2
#strength x2
#agility x1
# mijn 31 druid heeft 202 ap

In [6]:
mob_armor_health_df = pd.read_html('armor_health.html',
                                   skiprows = 6,
                                   index_col = 1)
mob_armor_health_df = mob_armor_health_df[0]

cols = ['Warrior','Paladin', 'Mage', 'Warrior.1', 'Paladin.1', 'Mage.1']
mob_armor_health_df = mob_armor_health_df.loc[:,cols]
mob_armor_health_df.columns = ['Warrior_Armor','Paladin_Armor', 'Mage_Armor', 'Warrior_Health', 'Paladin_Health', 'Mage_Health']
mob_armor_health_df.head()

,Warrior_Armor,Paladin_Armor,Mage_Armor,Warrior_Health,Paladin_Health,Mage_Health
Mob Level,,,,,,
1,20,NaN,NaN,42.0,NaN,NaN
2,21,NaN,NaN,55.0,NaN,NaN
3,46,NaN,NaN,71.0,NaN,69.0
4,82,NaN,NaN,86.0,NaN,83.0
5,126,105.0,NaN,102.0,NaN,98.0


In [7]:

#untalented
# avg total damage=449.41824620053745
# avg total damage=453.5262784531543
# avg total damage=452.1229344815227
# avg total damage=450.8468846672712

#5/5 ferocity
# avg total damage=464.35763373853
# avg total damage=462.21241725761234
# avg total damage=460.3423069908101
# avg total damage=461.4671121242105

#5/5 natural weapons
# avg total damage=483.39410705848974
# avg total damage=481.9524527187383
# avg total damage=481.74963492069924
# avg total damage=482.9941847349497

#thorns can't crit

In [ ]:
def non_binary_spell_cast():
    global non_binary_spell_modifier
    global mob_resist_rate
    global SpellMissChance
    randomroller()
    print(f'rolled {randomroll}')
    if randomroll <= SpellMissChance:
        print('missed spell')
        non_binary_spell_modifier = 0
    else:
        print('spell not missed')
        non_binary_spell_modifier = mob_resist_rate
        print(f'non_binary_spell_modifier= {non_binary_spell_modifier}')
        
non_binary_spell_cast()

In [84]:
#talents
natural_weapons_talentpoints = 5
natural_weapon_mult = 1 + 0.02 * natural_weapons_talentpoints
savage_fury_talentpoints = 0 #maul damage = (savage fury )*(natural weapons) *( base damage + maul bonus ) 
ferocity_talentpoints = 1
omen_of_clarity_talentpoints = 1
sharpened_claws_talentpoints = 0

current_energy = 0
current_level = 26
mob_level = 28
TargetLevel = mob_level
AttackerSkill = 5 * current_level #dit is een synoniem voor player attack rating
mob_defense_rating = 5 * mob_level
mob_wears_shield = True
NormalMissChance = 0.05
AttackerCrit = base_stats_df.at[current_level, 'Melee Crit'] + sharpened_claws_talentpoints * 0.02
mob_armor = mob_armor_health_df.at[mob_level, 'Warrior_Armor']
strength = base_stats_df.loc[current_level,'Strength']
agility = base_stats_df.loc[current_level,'Agility']
attack_power = current_level*2+2*strength+agility+ 90*(current_level/60)-20
auto_attack_damage_lowend = ((current_level*0.85) + (attack_power/14)) -10.1 *(current_level/60)
auto_attack_damage_highend = ((current_level*1.25) + (attack_power/14)) -10.1 *(current_level/60)
auto_attack_damage = (auto_attack_damage_lowend + auto_attack_damage_highend) /2
auto_attack_damage_final = 0.0
auto_attack_result = ''
randomroll = 0
print(f'auto_attack_damage = {auto_attack_damage}')
special_attack_result = ''
special_attack_result_modifier = 1
number_of_autoattacks = 0
totaldmg_autoattacks = 0.0
number_of_claw = 0
number_of_claw_hits_and_crits = 0
totaldmg_claw = 0.0
number_of_ooc_procs = 0
ooc_proc_active = False
total_number_of_rip_ticks = 0
number_of_rips_cast = 0
current_combopoints = 0
totaldmg_rip = 0
starttime_rip = 0
endtime_rip = 0
current_rip_dmg_per_tick = 0
non_binary_spell_modifier = 0 


#Claw rank damage bonus
if current_level < 28:
    claw_additional_damage = 27
elif current_level < 38:
    claw_additional_damage = 39
elif current_level < 48:
    claw_additional_damage = 57
elif current_level < 58:
    claw_additional_damage = 88
else:
    claw_additional_damage = 115
print(f'claw_additional_damage = {claw_additional_damage}')

#Rip rank damage bonus
if current_level < 28:
    rip_additional_damage = 42 + 0.24 * attack_power
elif current_level < 36:
    rip_additional_damage = 66 + 0.24 * attack_power
elif current_level < 44:
    rip_additional_damage = 90 + 0.24 * attack_power
elif current_level < 52:
    rip_additional_damage = 114 + 0.24 * attack_power
else:
    rip_additional_damage = 115 + 0.24 * attack_power
print(f'rip_additional_damage = {rip_additional_damage}')


#Spellhit
level_miss_chance = 0
if TargetLevel - current_level >= 5:
    level_miss_chance = 1 - 0.61
elif TargetLevel - current_level == 4:
    level_miss_chance = 1 - 0.72
elif TargetLevel - current_level == 3:
    level_miss_chance = 1 - 0.83
elif TargetLevel - current_level == 2:
    level_miss_chance = 1 - 0.94
elif TargetLevel - current_level == 1:
    level_miss_chance = 1 - 0.95
elif TargetLevel - current_level == 0:
    level_miss_chance = 1 - 0.96
elif TargetLevel - current_level == -1:
    level_miss_chance = 1 - 0.97
elif TargetLevel - current_level == -2:
    level_miss_chance = 1 - 0.98
elif TargetLevel - current_level == -3:
    level_miss_chance = 1 - 0.99
elif TargetLevel - current_level <= -4:
    level_miss_chance = 1 - 1
SpellMissChance = 0.01 + level_miss_chance
print(f'SpellMissChance = 0.01 + {level_miss_chance} = {SpellMissChance}')

#spell resistance
mob_base_resistance = 0
mob_base_resistance_rate = (0.75 * mob_base_resistance) / (5 * current_level)
mob_level_based_resistance_rate = max(0.02 * (TargetLevel - current_level), 0) #only for non-binary spells
mob_resist_rate = 1 - (mob_level_based_resistance_rate + mob_base_resistance_rate)

#Mob armor
X = (0.1 * mob_armor) / (8.5 * current_level + 40)
ArmorReductionFactor = 1 - (X / (1 + X))
print(f'ArmorReductionFactor = {ArmorReductionFactor}')

#Miss
if TargetLevel < 10:
    MissChance = NormalMissChance * (TargetLevel / 10)
elif (mob_defense_rating - AttackerSkill) <= 10:
    MissChance = 0.05 + (TargetLevel * 5 - AttackerSkill) * 0.001
elif (mob_defense_rating - AttackerSkill) >= 11:
    MissChance = 0.05 + (TargetLevel * 5 - AttackerSkill) * 0.002
print(f'MissChance = {MissChance}')

#Glancing
# glancing gebeurt alleen op white non-crits
GlancingChance = 0.10 + max(0, (TargetLevel - current_level) * 0.1)
LowEndGlancing  = min(0.91, 1.3 - 0.05*(mob_defense_rating - AttackerSkill))
HighEndGlancing = max(0.2, min(0.99, 1.2 - 0.03*(mob_defense_rating - AttackerSkill)))
GlancingReduction =  max(0, (HighEndGlancing + LowEndGlancing) / 2)
print(f'GlancingChance = {GlancingChance}')
print(f'GlancingReduction = {GlancingReduction}')

#Dodge
DodgeChance = 0.05 + (TargetLevel*5 - AttackerSkill) * 0.001
print(f'DodgeChance = {DodgeChance}')

#Block
if mob_wears_shield:
    BlockChance = max(0, min(0.05, 0.05 + (TargetLevel*5 - AttackerSkill) * 0.001))
else: BlockChance = 0
print(f'BlockChance = {BlockChance}')

#Crit
if (AttackerSkill - mob_defense_rating) < 0:
    CritChance = AttackerCrit + (AttackerSkill - mob_defense_rating) * 0.002
elif (mob_defense_rating - AttackerSkill) >= 0:
    CritChance = AttackerCrit + (AttackerSkill - mob_defense_rating) * 0.004
print(f'CritChance = {CritChance}')

#Parry
if (mob_level - current_level) < 3:
    ParryChance = 0.05 + 0.005 * (mob_level - current_level)
elif (mob_level - current_level) >= 3:
    ParryChance = 0.125 + 0.005 * (mob_level - current_level)
print(f'ParryChance = {ParryChance}')

#========================= Defs
def energy_regen(): 
    global current_energy
    global second
    if second % 2 == 0:
        if current_energy + 20 >= 100:
            current_energy = max(100, current_energy)
        elif current_energy + 20 < 100:
            current_energy += 20
    print(f'current_energy = {current_energy}')
    
def randomroller():
    global randomroll
    randomroll = round(np.random.randint(1,101) * 0.01, 2)

def OmenOfClarity():
    global number_of_ooc_procs
    global ooc_proc_active
    
    if omen_of_clarity_talentpoints:
        weapon_speed = 1
        PPM = 2
        proc_per_hit = weapon_speed * PPM / 60 
        randomroller()
        if randomroll <= proc_per_hit:
            print(f'OOC procced, because randomroll={randomroll} and proc_per_hit={proc_per_hit}')
            number_of_ooc_procs += 1
            ooc_proc_active = True

def autoattackdef():
    global auto_attack_result
    global auto_attack_damage_final
    global number_of_autoattacks
    global totaldmg_autoattacks
    
    randomroller()
    
    if randomroll <= MissChance:
        auto_attack_result = 'Miss'
        auto_attack_damage_final = 0
        
    elif randomroll <= MissChance + DodgeChance:
        auto_attack_result = 'Dodge'
        auto_attack_damage_final = 0
        
    elif randomroll <= MissChance + DodgeChance + ParryChance:
        auto_attack_result = 'Parry'
        auto_attack_damage_final = 0
        
    elif randomroll <= MissChance + DodgeChance + ParryChance + GlancingChance:
        auto_attack_result = 'Glancing Blow'
        auto_attack_damage_final = GlancingReduction * auto_attack_damage * ArmorReductionFactor
        OmenOfClarity()
        
    elif mob_wears_shield and randomroll <= MissChance + DodgeChance + ParryChance + GlancingChance + BlockChance:
        auto_attack_result = 'Block'
        auto_attack_damage_final = 0
        
    elif mob_wears_shield and randomroll <= MissChance + DodgeChance + ParryChance + GlancingChance + BlockChance + CritChance:
        auto_attack_result = 'Crit'
        auto_attack_damage_final = 2 * auto_attack_damage * ArmorReductionFactor 
        OmenOfClarity()
        
    elif randomroll < 1:
        auto_attack_result = 'Hit'
        auto_attack_damage_final = natural_weapon_mult * auto_attack_damage * ArmorReductionFactor 
        OmenOfClarity()
        
    number_of_autoattacks += 1
    totaldmg_autoattacks += auto_attack_damage_final
    print(f'autoattack {auto_attack_result} for {auto_attack_damage_final} based on roll of {randomroll}')

#autoattackdef()
#print(f' number_of_autoattacks={number_of_autoattacks} and totaldmg_autoattacks={totaldmg_autoattacks}')
# #white attack table
# print(f' Miss={MissChance}') #Miss
# print(f' Dodge={MissChance + DodgeChance}') #Dodge
# print(f' Parry={MissChance + DodgeChance + ParryChance}') #Parry
# print(f' Glancing Blow={MissChance + DodgeChance + ParryChance + GlancingChance}') #Glancing Blow
# print(f' Block={MissChance + DodgeChance + ParryChance + GlancingChance + BlockChance}')  # Block
# print(f' Crit={MissChance + DodgeChance + ParryChance + GlancingChance + BlockChance + CritChance}')    

def RipTickDmg():
    global current_rip_dmg_per_tick
    global totaldmg_rip
    global second
    global starttime_rip
    global endtime_rip
    global current_rips_remaining
    
    if starttime_rip <= second <= endtime_rip:
        if (second - starttime_rip) % 2 == 0:
            print(f'we will have a rip tick for {current_rip_dmg_per_tick} because second={second} is in between {starttime_rip} and {endtime_rip}')
            totaldmg_rip += current_rip_dmg_per_tick
            current_rips_remaining -= 1

def Rip():
    global number_of_rips_cast
    global current_combopoints
    global totaldmg_rip
    global current_energy
    global starttime_rip
    global endtime_rip
    global fight_duration
    global second
    global current_rip_dmg_per_tick
    global ooc_proc_active
    global total_number_of_rip_ticks
    global current_rips_remaining
    
    if ((current_energy >= 30) or ooc_proc_active) and current_combopoints > 1 and endtime_rip !=fight_duration and (fight_duration-second)>1: 
        if not ooc_proc_active:
            current_energy -= 30
        elif ooc_proc_active:
            ooc_proc_active = False
            print(f' used up OOC proc on Rip and set to false')
        specialattackdef()
        number_of_rips_cast += 1
        print(f'{current_combopoints}cp Rip {special_attack_result} for {special_attack_result_modifier * rip_additional_damage} based on special_attack_result_modifier={special_attack_result_modifier} and base rip damage for {rip_additional_damage}')
        if special_attack_result == 'Hit' or special_attack_result == 'Crit':
        
            
            current_combopoints = 0

            starttime_rip = second
            endtime_rip = min(second + 12, fight_duration)
            current_rips_remaining = (endtime_rip - starttime_rip) // 2
            print(f'current_rips_remaining={current_rips_remaining} and total_number_of_rip_ticks voor={total_number_of_rip_ticks}')
            total_number_of_rip_ticks += current_rips_remaining
            print(f'total_number_of_rip_ticks na={total_number_of_rip_ticks}')

            current_rip_dmg_per_tick = special_attack_result_modifier * rip_additional_damage / 6
#             totaldmg_rip += current_rips_remaining * (special_attack_result_modifier * rip_additional_damage)

            print(f'rip starts at {starttime_rip} and ends at {endtime_rip}')
        OmenOfClarity()

def Claw():
    global ferocity_talentpoints
    global number_of_claw
    global totaldmg_claw
    global current_energy
    global number_of_claw_hits_and_crits
    global ooc_proc_active
    
    if (current_energy >= 45 - ferocity_talentpoints) or ooc_proc_active: 
        if not ooc_proc_active:
            current_energy -= (45 - ferocity_talentpoints)
        elif ooc_proc_active:
            ooc_proc_active = False
            print(f' used up OOC proc on Claw and set to false')
        specialattackdef()
        print(f'Claw {special_attack_result} for {special_attack_result_modifier * claw_additional_damage} based on special_attack_result_modifier={special_attack_result_modifier} and base claw damage for {claw_additional_damage}')
        number_of_claw += 1
        OmenOfClarity()
        totaldmg_claw += special_attack_result_modifier * claw_additional_damage
        if special_attack_result == 'Crit' or special_attack_result == 'Hit':
            ComboPoints()

def specialattackdef():
    global special_attack_result
    global special_attack_result_modifier
    #First roll
    randomroller()
    #print(f'1st Roll={randomroll}')
       
    if randomroll <= MissChance:
        special_attack_result = 'Miss'
        special_attack_result_modifier = 0
    elif randomroll <= MissChance + DodgeChance:
        special_attack_result = 'Dodge'
        special_attack_result_modifier = 0
    elif randomroll <= MissChance + DodgeChance + ParryChance:
        special_attack_result = 'Parry'
        special_attack_result_modifier = 0
    elif mob_wears_shield and randomroll <= MissChance + DodgeChance + ParryChance +  BlockChance:
        special_attack_result = 'Block'
        special_attack_result_modifier = 0
    else:
        randomroller()
        #print(f'2nd Roll={randomroll}')
        
        if randomroll <= CritChance:
            special_attack_result = 'Crit'
            special_attack_result_modifier = 2 * natural_weapon_mult * ArmorReductionFactor
        else:
            special_attack_result = 'Hit'
            special_attack_result_modifier = 1 * natural_weapon_mult * ArmorReductionFactor


            
#specialattackdef()
#print(f'special attack type is {special_attack_result} based on special_attack_result_modifier={special_attack_result_modifier}')

# #yellow attack table
# print(f' Miss={MissChance}') #Miss
# print(f' Dodge={MissChance + DodgeChance}') #Dodge
# print(f' Parry={MissChance + DodgeChance + ParryChance}') #Parry
# print(f' Block={MissChance + DodgeChance + ParryChance + BlockChance}')  # Block
# print('-------------------------------')
# print(f' Crit={CritChance}')        

def ComboPoints():
    global current_combopoints
    
    if current_combopoints < 5:
        combopoints_before = current_combopoints
        current_combopoints += 1
        combopoints_after = current_combopoints
        print(f'current_combopoints went from {combopoints_before} to {combopoints_after}')
    else:
        print(f'already at cap of {current_combopoints} combopoints')

#========================= /Defs


iterations = 1000
fight_duration = 18
for iteration in range(iterations):
    endtime_rip = 0
    current_combopoints = 0
    print(f'-------------------------------------------------------------------')
    for second in range(1, fight_duration + 1):
        energy_regen()
        autoattackdef()
        RipTickDmg()
        Rip()
        Claw()
        
print(f'avg number_of_autoattacks = {number_of_autoattacks/iterations}')
print(f'avg number_of_claw={number_of_claw/iterations}')
print(f'avg number_of_rips_cast={number_of_rips_cast/iterations}')
print(f'avg number_of_ooc_procs={number_of_ooc_procs/iterations}')
print(f'avg total_number_of_rip_ticks={total_number_of_rip_ticks/iterations}')    

print(f'avg totaldmg_autoattacks = {totaldmg_autoattacks/iterations}')
print(f'avg totaldmg_rip={totaldmg_rip/iterations}')
print(f'avg totaldmg_claw={totaldmg_claw/iterations}')
print(f'avg total damage={(totaldmg_autoattacks+totaldmg_claw+totaldmg_rip)/iterations}')

# print(f'current_rip_dmg_per_tick={current_rip_dmg_per_tick}')
# print(f'endtime_rip={endtime_rip}')
# print(f'starttime_rip={starttime_rip}')

auto_attack_damage = 35.85190476190476
claw_additional_damage = 27
rip_additional_damage = 85.44
SpellMissChance = 0.01 + 0.06000000000000005 = 0.07000000000000005
ArmorReductionFactor = 0.6956289978678039
MissChance = 0.060000000000000005
GlancingChance = 0.30000000000000004
GlancingReduction = 0.85
DodgeChance = 0.060000000000000005
BlockChance = 0.05
CritChance = 0.162
ParryChance = 0.060000000000000005
-------------------------------------------------------------------
current_energy = 0
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.73
current_energy = 20
autoattack Miss for 0 based on roll of 0.03
current_energy = 20
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.51
current_energy = 40
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.51
current_energy = 40
autoattack Block for 0 based on roll of 0.51
current_energy = 60
autoattack Block for 0 based on roll of 1.0
Claw Hit for 20.660181236673775 based on special_attack_result_

current_energy = 34
autoattack Crit for 49.87924916235151 based on roll of 0.31
current_energy = 54
autoattack Hit for 27.43358703929333 based on roll of 0.39
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 0 to 1
current_energy = 10
OOC procced, because randomroll=0.01 and proc_per_hit=0.03333333333333333
autoattack Hit for 27.43358703929333 based on roll of 0.01
 used up OOC proc on Claw and set to false
Claw Parry for 0 based on special_attack_result_modifier=0 and base claw damage for 27
current_energy = 30
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.95
current_energy = 30
autoattack Dodge for 0 based on roll of 0.09
current_energy = 50
autoattack Hit for 27.43358703929333 based on roll of 0.83
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 1 to 2
current_energ

autoattack Hit for 27.43358703929333 based on roll of 0.92
current_energy = 26
autoattack Hit for 27.43358703929333 based on roll of 0.49
current_energy = 26
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.09
current_energy = 46
autoattack Crit for 49.87924916235151 based on roll of 0.96
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 0 to 1
current_energy = 2
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.84
current_energy = 22
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.91
current_energy = 22
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.22
current_energy = 42
autoattack Crit for 49.87924916235151 based on roll of 0.48
current_energy = 42
autoattack Hit for 27.43358703929333 based on roll of 0.61
current_energy = 62
autoattack Dodge for 0 based on roll of 0.11
Claw Block for 0 based on special_attack_result

total_number_of_rip_ticks na=167
rip starts at 14 and ends at 18
current_energy = 4
autoattack Crit for 49.87924916235151 based on roll of 0.62
current_energy = 24
autoattack Crit for 49.87924916235151 based on roll of 0.95
we will have a rip tick for 10.89633262260128 because second=16 is in between 14 and 18
current_energy = 24
autoattack Hit for 27.43358703929333 based on roll of 0.61
current_energy = 44
autoattack Dodge for 0 based on roll of 0.11
we will have a rip tick for 10.89633262260128 because second=18 is in between 14 and 18
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 0 to 1
-------------------------------------------------------------------
current_energy = 0
autoattack Crit for 49.87924916235151 based on roll of 0.44
current_energy = 20
autoattack Hit for 27.43358703929333 based on roll of 0.22
current_energy = 20
autoattack Parry for 0 based on roll of 0.15
current_e

Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 1 to 2
current_energy = 4
autoattack Crit for 49.87924916235151 based on roll of 0.93
current_energy = 24
autoattack Hit for 27.43358703929333 based on roll of 0.74
current_energy = 24
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.57
current_energy = 44
autoattack Hit for 27.43358703929333 based on roll of 0.86
2cp Rip Hit for 65.37799573560768 based on special_attack_result_modifier=0.7651918976545843 and base rip damage for 85.44
current_rips_remaining=3 and total_number_of_rip_ticks voor=209
total_number_of_rip_ticks na=212
rip starts at 12 and ends at 18
current_energy = 14
autoattack Parry for 0 based on roll of 0.14
current_energy = 34
autoattack Hit for 27.43358703929333 based on roll of 0.46
we will have a rip tick for 10.89633262260128 because second=14 is in between 12 and 18
current_energy = 34
autoattack Hit

...

autoattack Hit for 27.43358703929333 based on roll of 0.27
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 0 to 1
current_energy = 8
autoattack Hit for 27.43358703929333 based on roll of 0.11
current_energy = 28
autoattack Parry for 0 based on roll of 0.14
current_energy = 28
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.67
current_energy = 48
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.76
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 1 to 2
current_energy = 4
autoattack Hit for 27.43358703929333 based on roll of 0.55
current_energy = 24
autoattack Hit for 27.43358703929333 based on roll of 0.6
current_energy = 24
autoattack Block for 0 based on roll of 0.53
current_energy = 44
autoattack Crit for 49.87924916235151 based on roll of 0.05
2cp Rip

autoattack Crit for 49.87924916235151 based on roll of 0.13
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 1 to 2
current_energy = 6
autoattack Miss for 0 based on roll of 0.05
current_energy = 26
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.52
current_energy = 26
autoattack Hit for 27.43358703929333 based on roll of 0.5
current_energy = 46
autoattack Dodge for 0 based on roll of 0.11
2cp Rip Hit for 65.37799573560768 based on special_attack_result_modifier=0.7651918976545843 and base rip damage for 85.44
current_rips_remaining=3 and total_number_of_rip_ticks voor=2499
total_number_of_rip_ticks na=2502
rip starts at 12 and ends at 18
current_energy = 16
autoattack Crit for 49.87924916235151 based on roll of 0.86
current_energy = 36
autoattack Parry for 0 based on roll of 0.13
we will have a rip tick for 10.89633262260128 because second=14 is in between 12 and 18
cu

rip starts at 12 and ends at 18
current_energy = 4
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.07
current_energy = 24
autoattack Hit for 27.43358703929333 based on roll of 0.96
we will have a rip tick for 10.89633262260128 because second=14 is in between 12 and 18
current_energy = 24
autoattack Glancing Blow for 21.19868089399939 based on roll of 0.86
current_energy = 44
autoattack Hit for 27.43358703929333 based on roll of 0.39
we will have a rip tick for 10.89633262260128 because second=16 is in between 12 and 18
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 0 to 1
current_energy = 0
autoattack Hit for 27.43358703929333 based on roll of 0.66
current_energy = 20
autoattack Hit for 27.43358703929333 based on roll of 0.46
we will have a rip tick for 10.89633262260128 because second=18 is in between 12 and 18
--------------------------------------------------------

autoattack Glancing Blow for 21.19868089399939 based on roll of 0.55
Claw Hit for 20.660181236673775 based on special_attack_result_modifier=0.7651918976545843 and base claw damage for 27
current_combopoints went from 1 to 2
current_energy = 10
autoattack Glancing Blow for 21.19868089399939 based on roll of 1.0
current_energy = 30
autoattack Block for 0 based on roll of 0.52
2cp Rip Crit for 130.75599147121537 based on special_attack_result_modifier=1.5303837953091686 and base rip damage for 85.44
current_rips_remaining=5 and total_number_of_rip_ticks voor=2610
total_number_of_rip_ticks na=2615
rip starts at 8 and ends at 18
current_energy = 0
autoattack Miss for 0 based on roll of 0.01
current_energy = 20
autoattack Dodge for 0 based on roll of 0.1
we will have a rip tick for 21.79266524520256 because second=10 is in between 8 and 18
current_energy = 20
autoattack Parry for 0 based on roll of 0.18
current_energy = 40
autoattack Hit for 27.43358703929333 based on roll of 0.23
we will h

autoattack Miss for 0 based on roll of 0.02
we will have a rip tick for 10.89633262260128 because second=16 is in between 12 and 18
Claw Crit for 41.32036247334755 based on special_attack_result_modifier=1.5303837953091686 and base claw damage for 27
current_combopoints went from 0 to 1
current_energy = 12
autoattack Dodge for 0 based on roll of 0.1
current_energy = 32
autoattack Crit for 49.87924916235151 based on roll of 0.48
we will have a rip tick for 10.89633262260128 because second=18 is in between 12 and 18
-------------------------------------------------------------------
current_energy = 32
autoattack Crit for 49.87924916235151 based on roll of 0.18
current_energy = 52
autoattack Hit for 27.43358703929333 based on roll of 0.17
Claw Dodge for 0 based on special_attack_result_modifier=0 and base claw damage for 27
current_energy = 8
autoattack Crit for 49.87924916235151 based on roll of 0.4
current_energy = 28
autoattack Crit for 49.87924916235151 based on roll of 0.18
current_

level 20 test, 1000 iterations per keer, 20 seconden fight:  
https://classicdb.ch/?talent#0zLV0o  
avg total damage=495.84904525504555  
avg total damage=496.3233554543028  
avg total damage=490.1186092536928  
avg total damage=490.419443600198  
avg total damage=493.35048091867304  
avg total damage=491.521413205062 

https://classicdb.ch/?talent#0ZE00z  
avg total damage=469.60536778108207  
avg total damage=470.3019424180294  
avg total damage=468.67713137446356  
avg total damage=469.8835604524771  
avg total damage=471.0387953393395 

conclusie: op level 20 is het beter dat je natural weapons > ooc bent gegaan dan full feral

level 21 test (player en mob level ook 21 nu), 1000 iterations per keer, 20 seconden fight:  
https://classicdb.ch/?talent#0ZE00M  
avg total damage=500.80809050772757  
avg total damage=496.82136846348186  
avg total damage=498.5296860620977  
avg total damage=499.545095005327  
avg total damage=498.76305996801426  

https://classicdb.ch/?talent#0zLV0oZo  
avg total damage=511.86343686296675  
avg total damage=511.3034767323711  
avg total damage=512.6617021135  
avg total damage=513.6740494842375  
avg total damage=515.216063404812  

conclusie: op level 21 is het beter dat je natural weapons > ooc bent gegaan dan full feral

level 22 test (player en mob level ook 22 nu), 1000 iterations per keer, 20 seconden fight:  
https://classicdb.ch/?talent#0zLV0oZb  
avg total damage=539.4718155368879  
avg total damage=539.0400916631194  
avg total damage=539.6048825135542  
avg total damage=537.931288385405  
avg total damage=535.6836296996132  

https://classicdb.ch/?talent#0ZE00c  
avg total damage=529.1238945663637  
avg total damage=529.8980424644775  
avg total damage=529.3105151884452  
avg total damage=530.3454721678623  
avg total damage=530.6721833407895

conclusie: op level 22 is het beter dat je natural weapons > ooc bent gegaan dan full feral


https://www.youtube.com/watch?v=ivGeMsz_f4w&t=13279s 21 mob vs 21 mob duurt ongeveer 12-13 seconden voor Ebbnflow. Ga nu bij deze fight length proberen met level 20 of Rip waard is

https://classicdb.ch/?talent#0zLV0o met any cp rip > claw
avg total damage=333.65915839246344
avg total damage=329.7558811888153
avg total damage=331.25684170890946
avg total damage=332.7203872340704

https://classicdb.ch/?talent#0zLV0o met rip en ripticks uitgezet
avg total damage=317.6727283012795
avg total damage=317.529016379631
avg total damage=318.982360351264
avg total damage=319.2412588652787

https://classicdb.ch/?talent#0zLV0o met rip van minimaal 2cp > claw
avg total damage=318.389502634275
avg total damage=317.4986778791259
avg total damage=320.7427956771648
avg total damage=316.4483428571716
conclusie: bij een fight van maar 13 seconden is any cp rip beter dan geen rip, welke beter is dan 2cp rip

level 20, ooc enabled, fight duration 18 seconden. testen of 2cp rip al beter is dan geen rip
https://classicdb.ch/?talent#0zLV0o met any cp rip > claw
avg total damage=485.8089779129233
avg total damage=485.51953586631583
avg total damage=486.72844596425665

https://classicdb.ch/?talent#0zLV0o met rip en ripticks uitgezet
avg total damage=442.6683681189334
avg total damage=446.99619469779196
avg total damage=445.97107362383036

https://classicdb.ch/?talent#0zLV0o met rip van minimaal 2cp > claw
avg total damage=466.82674883490637
avg total damage=464.3862661263613
avg total damage=468.07971394804366
conclusie: bij een fight van 18 seconden is any cp rip beter dan 2cp rip, welke beter is dan geen rip